In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

#Configuration
spark = SparkSession\
    .builder\
    .master('local[2]')\
    .appName('quake_etl')\
    .config('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.12:2.4.1')\
    .getOrCreate()  



#load data
death = spark.read.csv(r"C:\Users\israt\OneDrive\Desktop\DM_Project\project\death.csv", header=True)



#Delete column
delete_columns = ['Cases - cumulative total per 100000 population', 'Cases - newly reported in last 7 days','Cases - newly reported in last 7 days per 100000 population','Cases - newly reported in last 24 hours','Deaths - cumulative total per 100000 population', 'Deaths - newly reported in last 7 days','Deaths - newly reported in last 7 days per 100000 population',
    'Deaths - newly reported in last 24 hours']

death = death.drop(*delete_columns)
death.show(8)



# count region
region = death.groupBy('WHO Region').count().withColumnRenamed('count','total region')

#Data
region.show(100)



#displaying variables types
death.printSchema()





#convert string to double
death = death.withColumn('Cases - cumulative total',death['Cases - cumulative total'].cast(DoubleType()))\
    .withColumn('Deaths - cumulative total',death['Deaths - cumulative total'].cast(DoubleType()))
death.printSchema()




#count all total
total_case = death.groupBy('WHO Region').sum('Cases - cumulative total').withColumnRenamed('sum(Cases - cumulative total)', 'Total Cases in Region')
total_death = death.groupBy('WHO Region').sum('Deaths - cumulative total').withColumnRenamed('sum(Deaths - cumulative total)', 'Total Death in Region')
total_case.show(20)
total_death.show(20)







# connect with mongodb by creating database

death.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1/covid_death.death').save()



# connect with mongodb by creating database
total_case.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1/covid_death.total_case').save()







# connect with mongodb by creating database
total_death.write.format('mongo')\
    .mode('overwrite')\
    .option('spark.mongodb.output.uri','mongodb://127.0.0.1/covid_death.total_death').save()

























+--------------------+---------------+------------------------+-------------------------+
|                Name|     WHO Region|Cases - cumulative total|Deaths - cumulative total|
+--------------------+---------------+------------------------+-------------------------+
|              Global|           null|               599071265|                  6467023|
|United States of ...|       Americas|                92953690|                  1033207|
|               India|South-East Asia|                44418585|                   527829|
|              Brazil|       Americas|                34384747|                   683494|
|              France|         Europe|                33466622|                   150451|
|             Germany|         Europe|                32095854|                   147223|
|  The United Kingdom|         Europe|                23493327|                   187761|
|   Republic of Korea|Western Pacific|                23142479|                    26689|
+---------